In [1185]:
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import seaborn as sns 
import matplotlib as plt 
%matplotlib inline
import glob
import csv
import tarfile
import numpy as np 
from tqdm import tqdm
import json
from progressive.bar import Bar
import pickle as pkl
import networkx as nx
from __future__ import unicode_literals

In [1186]:
# with open('Bipartite_Drug_graph_Normalized.pkl','rb') as f:
#     drug_association_graph  = pkl.load(f)
    
with open('Bipartite_Drug_graph.pkl','rb') as f:
    drug_association_graph  = pkl.load(f)

with open('Bipartite_Drug_category_graph.pkl','rb') as f:
    drug_cat_association_graph  = pkl.load(f)

In [1187]:
len(drug_association_graph.edges()), len(drug_cat_association_graph.edges())

(7954, 33493)

In [1188]:
drug_cat_association_graph.nodes(data=True)

[(u'Vitamins', {'Id': u'/categories/DBCAT000230', 'type': 'category'}),
 (u'Prostacyclin Analogues',
  {'Id': u'/categories/DBCAT002688', 'type': 'category'}),
 ('Pravastatin Sodium', {'Id': '0212000X0', 'type': 'drug'}),
 (u'UGT1A1 Substrates',
  {'Id': u'/categories/DBCAT003871', 'type': 'category'}),
 ('Clindamycin/Tretinoin', {'Id': '1306010AB', 'type': 'drug'}),
 (u'Lactates', {'Id': u'/categories/DBCAT001522', 'type': 'category'}),
 (u'Caffeine', {'Id': u'/categories/DBCAT004728', 'type': 'category'}),
 (u'BCRP/ABCG2 Inhibitors',
  {'Id': u'/categories/DBCAT002662', 'type': 'category'}),
 ('Oxycodone Hydrochloride', {'Id': '0407020AD', 'type': 'drug'}),
 (u'Benzoquinones', {'Id': u'/categories/DBCAT001600', 'type': 'category'}),
 ('Floxuridine', {'Id': '0801030G0', 'type': 'drug'}),
 (u'Cytochrome P-450 CYP2B6 Inhibitors (strength unknown)',
  {'Id': u'/categories/DBCAT004517', 'type': 'category'}),
 (u'Sucrose-specific Enzyme',
  {'Id': u'/categories/DBCAT003477', 'type': 'categ

In [1189]:
files = glob.glob('../BL_Work/openPrescribe/uncompressed/*.tar.gz')
print files

[u'../BL_Work/openPrescribe/uncompressed/T201010PDPI BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201902PDPI BNFT.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201902CHEM SUBS.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201008ADDR BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201601CHEM SUBS.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201601PDPI BNFT.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201011PDPI BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201901ADDR BNFT.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201009PDPI BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201011ADDR BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201010ADDR BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201010CHEM SUBS.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201602ADDR BNFT.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201603PDPI BNFT.CSV.tar.gz', u'../BL_Work/openPr

In [1190]:
#Different functions to extract different data form the prescription 

def cleanStringofUTF(string):
    cleaned = string.replace('\xe8','e').replace('\xf6','o')
    return cleaned

def enrichdrugs(chem_dict , drugs):
    diabetes_drug_words = [drugs[k]['name'].lower() for k in drugs]
    for drug in chem_dict:
        Name = chem_dict[drug]['name'].replace('(','').replace(')','')
        slot1 = Name.lower().split('/')
        slot2 = Name.lower().split(' ')
        slot3 = Name.lower().split(' & ')
        common1 = set(diabetes_drug_words).intersection(slot1)
        common2 = set(diabetes_drug_words).intersection(slot2)
        common3 = set(diabetes_drug_words).intersection(slot3)
        
        if len(common1) > 0 or len(common2) > 0 or len(common3) > 0:
            print common1 , common2 , common3
            drugs[chem_dict[drug]['code']] = {'disease':'' , 'disease_given_drug':0.0 , 'matched_disease':'', 'name':chem_dict[drug]['name'].strip() }

            
            
def makeChemDict(BNF_Chem):
    chem_dict = {}
    for index, row in BNF_Chem.iterrows():
        chem_dict[row['UNII_drugbank']] = {}
        chem_dict[row['UNII_drugbank']]['name'] = row['NAME']
        chem_dict[row['UNII_drugbank']]['code'] = row['BNF_code']
    return chem_dict
    
def getDrugCategory(categorylist, BNF_Chem, drugbankDict):
    allMatched = []
    drugs = {}
    chem_dict = makeChemDict(BNF_Chem)
    
    for k in drugbankDict:
        if len(drugbankDict[k]['Categories']) > 0:
            for cat in drugbankDict[k]['Categories']:
                matched_memo = []
                catString = cat.values()[0]#.split('\u2014')[-1]
                t = catString.lower().strip()
                for categoryString in categorylist:
                    categoryString = categoryString.lower()
                    if t.find(categoryString) >= 0:
                        matched_memo.append(categoryString)
                if k in chem_dict:
                    if len(matched_memo) > 0:# == len(categorylist):
                        allMatched.append(k)
                        print chem_dict[k]
                        drugs[chem_dict[k]['code']] = {}
                        drugs[chem_dict[k]['code']]['name'] = chem_dict[k]['name']
                        drugs[chem_dict[k]['code']]['matched_cat'] = categorylist
    enrichdrugs(chem_dict,drugs)               
    return list(set(allMatched)) , drugs


def getDrugforDiseaseDrugbank(categorylist, BNF_Chem, drugbankDict):
    allMatched = []
    drugs = {}
    chem_dict = makeChemDict(BNF_Chem)
    
    for k in drugbankDict:
        if len(drugbankDict[k]['Associations']) > 0:
            for cat in drugbankDict[k]['Associations']:
                matched_memo = []
                catString = cat.values()[0]
                t = catString.lower().strip()
                for categoryString in categorylist:
                    categoryString = categoryString.lower()
                    if t.find(categoryString) >= 0:
                        matched_memo.append(categoryString)
                if k in chem_dict:
                    if len(matched_memo) > 0:
                        allMatched.append(k)
                        print chem_dict[k]
                        drugs[chem_dict[k]['code']] = {}
                        drugs[chem_dict[k]['code']]['name'] = chem_dict[k]['name']
                        drugs[chem_dict[k]['code']]['matched_cat'] = categorylist
    enrichdrugs(chem_dict,drugs)               
    return  drugs


def findDrugsForDisease(Graph, Disease, BNF_Chem ):#,threshProb):
    chem_dict = makeChemDict(BNF_Chem)
    drugs = {}
    for e in Graph.edges(data=True):
        if (cleanStringofUTF(e[1]).lower().find(Disease.lower()) >=0) or (cleanStringofUTF(e[0]).lower().find(Disease.lower()) >= 0) :
            drugNode = ''
            matchedDisease = ''
            if Graph.node[e[0]]['type'] == 'symptom':
                drugNode = e[1]
                matchedDisease = e[0]
            else:
                drugNode = e[0]
                matchedDisease = e[1]
            drugs[Graph.node[drugNode]['Id']] = {}
            drugs[Graph.node[drugNode]['Id']]['name'] = drugNode
            drugs[Graph.node[drugNode]['Id']]['matched_disease'] = matchedDisease
            drugs[Graph.node[drugNode]['Id']]['disease'] = Disease
    enrichdrugs(chem_dict,drugs)
    return drugs




def findDrugsForCategory(Graph, Cat, BNF_Chem ):#,threshProb):
    chem_dict = makeChemDict(BNF_Chem)
    drugs = {}
    for e in Graph.edges(data=True):
        if (cleanStringofUTF(e[1]).lower().find(Cat.lower()) >=0) or (cleanStringofUTF(e[0]).lower().find(Cat.lower()) >= 0) :
            drugNode = ''
            matchedDisease = ''
            if Graph.node[e[0]]['type'] == 'category':
                drugNode = e[1]
                matchedDisease = e[0]
            else:
                drugNode = e[0]
                matchedDisease = e[1]
            print Graph.node[drugNode]['Id']
            drugs[Graph.node[drugNode]['Id']] = {}
            drugs[Graph.node[drugNode]['Id']]['name'] = drugNode
            drugs[Graph.node[drugNode]['Id']]['matched_cat'] = matchedDisease
            drugs[Graph.node[drugNode]['Id']]['category'] = Cat
    enrichdrugs(chem_dict,drugs)
    return drugs

def findDrugByName(Graph, name):
    foundDrugs = []
    for node in Graph.nodes(data=True):
        if node[1]['type'] == 'drug':
            if name.lower() in node[0].lower().split(' '):
                foundDrugs.append(node)
    return foundDrugs

def relabelNodes(graph):
    mapping = {}
    for node in graph.nodes():
        mapping[node] = cleanStringofUTF(node)
        
    return nx.relabel_nodes(graph,mapping)


def readChem(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('CHEM') > 0:
                df = pd.read_csv(f,compression='infer',header=None,skiprows=1)
                return df

def readAddress(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('ADDR') > 0:
                df = pd.read_csv(f,compression='infer',header=None,skiprows=1)
                return df

def readPrescriptions(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('PDPI') > 0:
                df = pd.read_csv(f,compression='infer',header=None,skiprows=1)
                return df



In [1191]:
# # chem = readChem('201601',files)
# chem = pd.read_csv('../BL_Work/CHEM_MASTER_MAP.csv')
# len(chem)

# # chem = chem.dropna()

In [1192]:
drugbank_dict = json.load(open('Drugbank_drugs_data.json','rb'))

In [1193]:
condition_Association_map = {}
for k in drugbank_dict:
    if len(drugbank_dict[k]['Associations']) > 0:
        for ass in drugbank_dict[k]['Associations']:
            
            disease = ass[ass.keys()[0]]
            if disease not in condition_Association_map:
                condition_Association_map[disease] = []
            condition_Association_map[disease].append(drugbank_dict[k]['Name'])
    

In [1194]:
len(condition_Association_map)

2955

In [1195]:
sorted_Disease = sorted(condition_Association_map.items(), key=lambda item: len(item[1]),reverse=True)
df = pd.DataFrame(sorted_Disease)
df.rename(columns={0: 'Condition', 1: 'Drugs'}, inplace=True)
# df.head(n=50)
# df.to_csv('Sorted_disease_drug_list.csv', encoding = 'utf-8')

In [1196]:
drugbank_dict['WK2XYI10QM']

{u'ATC_Codes': [{u'/atc/G02CC01': u'G02CC01 \u2014 Ibuprofen'},
  {u'/atc/G02CC': u'G02CC \u2014 Antiinflammatory products for vaginal administration'},
  {u'/atc/G02C': u'G02C \u2014 OTHER GYNECOLOGICALS'},
  {u'/atc/G02': u'G02 \u2014 OTHER GYNECOLOGICALS'},
  {u'/atc/G': u'G \u2014 GENITO URINARY SYSTEM AND SEX HORMONES'},
  {u'/atc/C01EB16': u'C01EB16 \u2014 Ibuprofen'},
  {u'/atc/C01EB': u'C01EB \u2014 Other cardiac preparations'},
  {u'/atc/C01E': u'C01E \u2014 OTHER CARDIAC PREPARATIONS'},
  {u'/atc/C01': u'C01 \u2014 CARDIAC THERAPY'},
  {u'/atc/C': u'C \u2014 CARDIOVASCULAR SYSTEM'},
  {u'/atc/M01AE01': u'M01AE01 \u2014 Ibuprofen'},
  {u'/atc/M01AE': u'M01AE \u2014 Propionic acid derivatives'},
  {u'/atc/M01A': u'M01A \u2014 ANTIINFLAMMATORY AND ANTIRHEUMATIC PRODUCTS, NON-STEROIDS'},
  {u'/atc/M01': u'M01 \u2014 ANTIINFLAMMATORY AND ANTIRHEUMATIC PRODUCTS'},
  {u'/atc/M': u'M \u2014 MUSCULO-SKELETAL SYSTEM'},
  {u'/atc/M02AA13': u'M02AA13 \u2014 Ibuprofen'},
  {u'/atc/M02AA':

In [1197]:
chem = pd.read_csv('../BL_Work/CHEM_MASTER_MAP.csv')

In [1198]:
chem.head()

,Unnamed: 0,BNF_code,NAME,UNII_drugbank
0,0,0101010A0,Alexitol Sodium,[]
1,1,0101010B0,Almasilate,[]
2,2,0101010C0,Aluminium Hydroxide,[]
3,3,0101010D0,Aluminium Hydroxide With Magnesium,[]
4,4,0101010E0,Hydrotalcite,17432CG1KU


In [1199]:
opiod_matched , drugs_opiates = getDrugCategory(['opioid'],chem,drugbank_dict)

{u'code': '0410030F0', u'name': 'Levacetylmethadol Hydrochloride                             '}
{u'code': '0410030F0', u'name': 'Levacetylmethadol Hydrochloride                             '}
{u'code': '0407020K0', u'name': 'Diamorphine Hydrochloride (Systemic)                        '}
{u'code': '0407020K0', u'name': 'Diamorphine Hydrochloride (Systemic)                        '}
{u'code': '0410030A0', u'name': 'Buprenorphine Hydrochloride                                 '}
{u'code': '0410030A0', u'name': 'Buprenorphine Hydrochloride                                 '}
{u'code': '0410030A0', u'name': 'Buprenorphine Hydrochloride                                 '}
{u'code': '0410030A0', u'name': 'Buprenorphine Hydrochloride                                 '}
{u'code': '0106060A0', u'name': 'Methylnaltrexone Bromide                                    '}
{u'code': '0106060A0', u'name': 'Methylnaltrexone Bromide                                    '}
{u'code': '0407020C0', u'name': 'Codeine

In [1200]:
antiBio_matched , drugs_antiBio = getDrugCategory(['antibiotic','Anti-Bacterial Agents'],chem,drugbank_dict)

{u'code': '0504050T0', u'name': 'Paromomycin Sulfate                                         '}
{u'code': '0802020U0', u'name': 'Sirolimus                                                   '}
{u'code': '0802020U0', u'name': 'Sirolimus                                                   '}
{u'code': '1103010Z0', u'name': 'Vancomycin Hydrochloride                                    '}
{u'code': '1310012C0', u'name': 'Chlortetracycline Hydrochloride                             '}
{u'code': '1310012C0', u'name': 'Chlortetracycline Hydrochloride                             '}
{u'code': '050102020', u'name': 'Cefpirome Sulfate                                           '}
{u'code': '0501012U0', u'name': 'Temocillin Sodium                                           '}
{u'code': '0501120Y0', u'name': 'Moxifloxacin                                                '}
{u'code': '0501015P0', u'name': 'Pivmecillinam Hydrochloride                                 '}
{u'code': '0501050S0', u'name': 'Spiramy

In [1201]:
antiDM_matched , drugs_antiDM = getDrugCategory(['diabetes','Type 2'],chem,drugbank_dict)

{u'code': '0601023I0', u'name': 'Guar Gum                                                    '}
{u'code': '0601022B0', u'name': 'Metformin Hydrochloride                                     '}
{u'code': '0601023AQ', u'name': 'Dulaglutide                                                 '}
{u'code': '0601021X0', u'name': 'Tolbutamide                                                 '}
{u'code': '0601021X0', u'name': 'Tolbutamide                                                 '}
{u'code': '0601011P0', u'name': 'Insulin Glulisine                                           '}
{u'code': '0601023AB', u'name': 'Liraglutide                                                 '}
{u'code': '0601021A0', u'name': 'Glimepiride                                                 '}
{u'code': '0205052R0', u'name': 'Telmisartan with Diuretic                                   '}
{u'code': '0601012X0', u'name': 'Insulin Detemir                                             '}
{u'code': '0601021R0', u'name': 'Gliquid

In [1202]:
len(drugs_opiates)

32

In [1203]:
# chem.head(n=100)

In [1204]:
opiod_BNF = chem[chem['UNII_drugbank'].isin(opiod_matched)]

In [1205]:
antiBio_BNF = chem[chem['UNII_drugbank'].isin(antiBio_matched)]

In [1206]:
antiDM_BNF = chem[chem['UNII_drugbank'].isin(antiDM_matched)]

In [1207]:
len(antiBio_BNF)

252

In [1208]:
# antiBio_BNF.head(n=100)

In [1209]:
pd.set_option('display.max_columns', None)  
len(opiod_BNF)

59

In [1210]:
# name_Key_Dict = {k : drugbank_dict[k]['Name'] for k in drugbank_dict}
# for index , row in chem.iterrows():
#     for k in name_Key_Dict:
#         drug = row[1]
#         slot1 =[word.lower().strip() for word in  drug.lower().split('/')]
#         slot2 =[word.lower().strip() for word in  drug.lower().split(' ')]
#         slot3 =[word.lower().strip() for word in drug.lower().split('&')]
#         if name_Key_Dict[k].lower().strip() in slot1 or name_Key_Dict[k].lower().strip() in slot2 or name_Key_Dict[k].lower().strip() in slot3:
#             row[2] = k
            

In [1211]:
# name_Key_Dict

In [1212]:
# chem.to_csv('../BL_Work/CHEM_MASTER_MAP.csv')

In [1213]:
drug_association_graph = relabelNodes(drug_association_graph)

In [1214]:
# chem.rename(columns={0:'BNF_code',
#                           1:'NAME',
#                           2:'UNII_drugbank'}, 
#                  inplace=True)

In [1215]:
# drug_association_graph.nodes(data=True)

In [1216]:
len([k for k in drug_association_graph.nodes(data=True) if k[1]['type'] == 'symptom'])

2662

In [1217]:
# drug_cat_association_graph.nodes(data=True)

In [1218]:
# print [node[1]['Id'] for node in drug_cat_association_graph.nodes(data=True) if node[1]['type'] == 'drug']

In [1219]:
# db_opioid = findDrugsForCategory(drug_cat_association_graph,"Opioid",chem )

In [1220]:
# db_opioid.keys()

In [1221]:
# db_opioid

In [1222]:
opiod_manual = {'0407010A0':'Combination',
'0407010F0':'Combination',
'0407010M0':'Combination',
'0407010M0':'Combination',
'0407010N0':'Combination',
'0407010N0':'Combination',
'0407010Q0':'Combination',
'0407010W0':'Combination',
'0407010X0':'Combination',
'040702020':'Morphine',
'040702040':'Tramadol',
'040702050':'Hydromorphone',
'0407020A0':'Fentanyl',
'0407020AB':'Papaveretum',
'0407020AD':'Oxycodone',
'0407020AF':'Combination',
'0407020AG':'Tapentadol',
'0407020B0':'Buprenorphine',
'0407020C0':'Codeine',
'0407020D0':'Dextromoramide ',
'0407020E0':'Dextropropoxyphene',
'0407020G0':'Dihydrocodeine ',
'0407020H0':'Dipipanone ',
'0407020K0':'Diamorphine ',
'0407020L0':'Meptazinol ',
'0407020M0':'Methadone ',
'0407020P0':'Morphine ',
'0407020Q0':'Morphine ',
'0407020T0':'Pentazocine ',
'0407020U0':'Pentazocine ',
'0407020V0':'Pethidine ',
'0407020W0':'Opium',
'0407020X0':'Phenazocine ',
'0407020Y0':'Nalbuphine ',
'0407020Z0':'Oxycodone',
'1001010J0':'Combination',}

In [1223]:
# chem

In [1224]:
opiod_BNF_manual = chem[chem['BNF_code'].isin(opiod_manual.keys())]

In [1225]:
len(opiod_BNF_manual)

34

In [1226]:
intersect =  list(set(opiod_manual.keys()).intersection(opiod_BNF['BNF_code']))

In [1227]:
missing = [k for k in opiod_manual.keys()if k not in intersect]

In [1228]:
 chem[chem['BNF_code'].isin(missing)]

,Unnamed: 0,BNF_code,NAME,UNII_drugbank
952,952,0407010A0,Aspirin & Papaveretum,[]
955,955,0407010F0,Co-Codamol (Codeine Phos/Paracetamol),[]
959,959,0407010M0,Co-Codaprin (Codeine Phos/Aspirin),[]
960,960,0407010N0,Co-Dydramol (Dihydrocodeine/Paracet),[]
962,962,0407010Q0,Co-Proxamol (Dextroprop HCl/Paracet),[]
968,968,0407010W0,Aspirin Combined Preparations,[]
969,969,0407010X0,Paracetamol Combined Preparations,[]
975,975,0407020AB,Papaveretum,[]
987,987,0407020H0,Dipipanone Hydrochloride,[]
997,997,0407020V0,Pethidine Hydrochloride,[]


In [1229]:
len(opiod_BNF)

59

In [1230]:
opiod_BNF

,Unnamed: 0,BNF_code,NAME,UNII_drugbank
88,88,0104020D0,Codeine Phosphate Compound Mixtures,UX6OWY2V7J
90,90,0104020L0,Loperamide Hydrochloride,6X9OC3H4II
91,91,0104020N0,Opium & Morphine,76I7G6D29C
92,92,0104020P0,Loperamide Hydrochloride & Simeticone,6X9OC3H4II
154,154,0106060A0,Methylnaltrexone Bromide,0RK7M7IABE
155,155,0106060B0,Naloxegol,44T7335BKE
691,691,0309010C0,Codeine Phosphate,UX6OWY2V7J
693,693,0309010N0,Diamorphine Hydrochloride,70D95007SX
695,695,0309010S0,Methadone Hydrochloride,UC6VBE7V1Z
699,699,0309020AA,Ibuprofen/Pseudoephed HCl,WK2XYI10QM


In [1231]:
import re 
s = "Doxycycline Hyclate_Tab Disper 100mcg S/F"
array = re.findall(r'([0-9]{1,})([mg])', s) 
print array

[(u'100', u'm')]


In [1239]:
missed= [k for k in opiod_manual if k not in list(opiod_BNF['BNF_code'])]

In [1240]:
extra = [k for k in list(opiod_BNF['BNF_code']) if k not in opiod_manual]

In [1241]:
# opiod_BNF['NAME']

In [1242]:
chem[chem['BNF_code'].isin(extra)]['NAME']

88      Codeine Phosphate Compound Mixtures                         
90      Loperamide Hydrochloride                                    
91      Opium & Morphine                                            
92      Loperamide Hydrochloride & Simeticone                       
154     Methylnaltrexone Bromide                                    
155     Naloxegol                                                   
691     Codeine Phosphate                                           
693     Diamorphine Hydrochloride                                   
695     Methadone Hydrochloride                                     
699     Ibuprofen/Pseudoephed HCl                                   
701     Guaiacol & Codeine                                          
718     Powdered Opium                                              
951     Paracetamol & Ibuprofen                                     
965     Aspirin Paracetamol & Codeine                               
967     Paracetamol & Codeine Phos

In [1243]:
db_diabetes = findDrugsForDisease(drug_association_graph,"depression",chem )

set([]) set([u'amoxapine']) set([])
set([]) set([u'aripiprazole']) set([])
set([]) set([u'perphenazine']) set([])
set([]) set([u'lamotrigine']) set([])
set([]) set([u'isocarboxazid']) set([])
set([]) set([u'oxazepam']) set([])
set([]) set([u'pindolol']) set([])
set([]) set([u'mirtazapine']) set([])
set([]) set([u'chlordiazepoxide']) set([])


In [1244]:
db_diabetes_drugBank = getDrugforDiseaseDrugbank(['Depressive','depression'],chem,drugbank_dict)

{u'code': '0403010V0', u'name': 'Nortriptyline                                               '}
{u'code': '0403010V0', u'name': 'Nortriptyline                                               '}
{u'code': '0410020A0', u'name': 'Bupropion Hydrochloride                                     '}
{u'code': '0402020AC', u'name': 'Olanzapine Embonate                                         '}
{u'code': '0402020AC', u'name': 'Olanzapine Embonate                                         '}
{u'code': '0402020AC', u'name': 'Olanzapine Embonate                                         '}
{u'code': '0305010W0', u'name': 'Doxapram Hydrochloride                                      '}
{u'code': '0305010W0', u'name': 'Doxapram Hydrochloride                                      '}
{u'code': '0403010C0', u'name': 'Amoxapine                                                   '}
{u'code': '0403010C0', u'name': 'Amoxapine                                                   '}
{u'code': '0403010C0', u'name': 'Amoxapi

In [1245]:
len(db_diabetes)

29

In [1246]:
len(db_diabetes_drugBank)

47

In [1247]:
%store -r diabetes_drugs

In [1248]:
missed_drugs = [diabetes_drugs[k]['ChemName'] for k in diabetes_drugs if k not in db_diabetes_drugBank]
print missed_drugs

['Tryptophan', 'Citalopram Hydrobromide', 'Agomelatine', 'Reboxetine', 'Duloxetine Hydrochloride (old)', 'Oxitriptan', 'Flupentixol Hydrochloride', 'Duloxetine Hydrochloride']


In [1023]:
diabetes_drugs

{'0403020H0': {'BNF_Classification': '4.3.2',
  'ChemName': 'Isocarboxazid',
  'category': 'Monoamine-Oxidase Inhibitors (Maois)',
  'disease': 'depression'},
 '0403020K0': {'BNF_Classification': '4.3.2',
  'ChemName': 'Moclobemide',
  'category': 'Monoamine-Oxidase Inhibitors (Maois)',
  'disease': 'depression'},
 '0403020M0': {'BNF_Classification': '4.3.2',
  'ChemName': 'Phenelzine Sulfate',
  'category': 'Monoamine-Oxidase Inhibitors (Maois)',
  'disease': 'depression'},
 '0403020Q0': {'BNF_Classification': '4.3.2',
  'ChemName': 'Tranylcypromine Sulfate',
  'category': 'Monoamine-Oxidase Inhibitors (Maois)',
  'disease': 'depression'},
 '0403030D0': {'BNF_Classification': '4.3.3',
  'ChemName': 'Citalopram Hydrobromide',
  'category': 'Selective Serotonin Re-Uptake Inhibitors',
  'disease': 'depression'},
 '0403030E0': {'BNF_Classification': '4.3.3',
  'ChemName': 'Fluoxetine Hydrochloride',
  'category': 'Selective Serotonin Re-Uptake Inhibitors',
  'disease': 'depression'},
 '04

In [232]:
# db_diabetes.keys()

In [253]:
# [k for k in diabetes_drugs if k not in db_diabetes]

In [254]:
# [k for k in diabetes_drugs if k not in db_diabetes_drugBank]

In [234]:
# diabetes_drugs['0402010AF']

In [238]:
len([db_diabetes[k] for k in db_diabetes if k not in ])

10

In [255]:
diabetes_drugs

{'0601011A0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Insulin Aspart',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601011L0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Insulin Lispro',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601011N0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Soluble Insulin (Neutral Insulin)',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601011P0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Insulin Glulisine',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601011R0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Insulin Human',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601012D0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Biphasic Isophane Insulin',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601012F0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Biphasic Insulin Lispro',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601012G0': {'BNF_Classification': '6.1.1',
  'ChemNam

In [256]:
# DiseaseDrugs = json.load(open('../processed/DiseasesMatched.json','rb'))

In [270]:
diseases = ["psoriasis",
 "gastroparesis",
 "multiple sclerosis",
 "crohn's disease",
 "interstitial cystitis",
 "rheumatoid",
 "parkinson",
 "dysautonomia",
 "kidney stones",
 "chronic fatigue syndrome",
 "bipolar",
 "hypothyroidism",
 "sleep apnea",
 "Meniere's disease",
 "dementia",
 "irritable bowel syndrome",
 "depression",
 "diabetes"]

In [470]:
DiseaseDrugs = {}
for d in diseases:
    drugs = findDrugsForDisease(drug_association_graph,d ,chem)
    for drug in drugs:
        DiseaseDrugs[drug] = {}
        DiseaseDrugs[drug]['chemName'] = drugs[drug]['name']
        DiseaseDrugs[drug]['disease'] = d

set([]) set([u'calcipotriol']) set([])
set([]) set([u'dexamethasone']) set([])
set([]) set([u'acitretin']) set([])
set([]) set([u'pimecrolimus']) set([])
set([]) set([u'methylprednisolone']) set([])
set([]) set([u'betamethasone']) set([])
set([]) set([u'tacrolimus']) set([])
set([]) set([u'infliximab']) set([])
set([]) set([u'prednisolone']) set([])
set([]) set([u'desoximetasone']) set([])
set([]) set([u'adalimumab']) set([])
set([]) set([u'tazarotene']) set([])
set([]) set([u'ciclosporin']) set([])
set([]) set([u'etanercept']) set([])
set([]) set([u'prednisone']) set([])
set([]) set([u'apremilast']) set([])
set([]) set([u'calcitriol']) set([])
set([]) set([u'azathioprine']) set([])
set([]) set([u'methoxsalen']) set([])
set([]) set([u'hydrocortisone']) set([])
set([]) set([u'methotrexate']) set([])
set([]) set([u'domperidone']) set([])
set([]) set([u'erythromycin']) set([])
set([]) set([u'daclizumab']) set([])
set([]) set([u'tetracosactide']) set([])
set([]) set([u'alemtuzumab']) set([

In [ ]:
DiseaseDrugsAll = {}
for d in diseases:
    drugs = findDrugsForDisease(drug_association_graph,d ,chem)
    for drug in drugs:
        DiseaseDrugs[drug] = {}
        DiseaseDrugs[drug]['chemName'] = drugs[drug]['name']
        DiseaseDrugs[drug]['disease'] = d

In [472]:
diabetes = {k:DiseaseDrugs[k] for k in DiseaseDrugs if DiseaseDrugs[k]['disease'] == 'diabetes'}

In [474]:
diabetes_metformin = {k:DiseaseDrugs[k] for k in DiseaseDrugs if DiseaseDrugs[k]['chemName'].find('Metformin') >= 0}

In [486]:
diabetes_metformin

{'0601022B0': {u'chemName': u'Metformin Hydrochloride',
  u'disease': u'diabetes'},
 '0601023AD': {u'chemName': u'Metformin Hydrochloride/Sitagliptin',
  u'disease': u'diabetes'},
 '0601023AF': {u'chemName': 'Linagliptin/Metformin', u'disease': u'diabetes'},
 '0601023AH': {u'chemName': 'Saxagliptin/Metformin', u'disease': u'diabetes'},
 '0601023AJ': {u'chemName': u'Alogliptin/Metformin', u'disease': u'diabetes'},
 '0601023AL': {u'chemName': 'Dapagliflozin/Metformin',
  u'disease': u'diabetes'},
 '0601023AP': {u'chemName': 'Canagliflozin/Metformin',
  u'disease': u'diabetes'},
 '0601023AR': {u'chemName': 'Empagliflozin/Metformin',
  u'disease': u'diabetes'},
 '0601023V0': {u'chemName': 'Metformin Hydrochloride/Rosiglitazone',
  u'disease': u'diabetes'},
 '0601023W0': {u'chemName': 'Metformin Hydrochloride/Pioglitazone',
  u'disease': u'diabetes'},
 '0601023Z0': {u'chemName': 'Metformin Hydrochloride/Vildagliptin',
  u'disease': u'diabetes'}}

In [954]:
json.dump(DiseaseDrugs,open('Disease_Drug_DrugBank.json','wb'))

In [274]:
SymptomDrugs = json.load(open('SymptomsMatched.json','rb'))

In [275]:
symptoms = [k.replace('+',' ').replace('"','').lower() for k in SymptomDrugs.keys()]
print symptoms

[u'impairment level of vision', u'backache', u'dyssomnia', u'loss of hair', u'mood swings', u'tired', u'weight gain', u'emotional upset', u'disorientated', u'clouded consciousness', u'stomach ache', u'increased thirst', u'sleep deprivation', u'heartburn', u'abdominal pain', u'swelling', u'feeling agitated', u'impairment of balance', u'paresthesia', u'blurring of visual image', u'feeling angry', u'loss of appetite', u'pain', u'tremor', u'rectal hemorrhage', u'abnormal gait', u'impotence', u'eruption of skin', u'feeling hopeless', u'indigestion', u"parkinson's disease", u'trigeminal neuralgia', u'bowel dysfunction', u'parkinsonism', u'delusional disorder', u'muscle pain', u'restlessness', u'malaise', u'thoughts of self harm', u'dementia', u'depressive disorder', u'hypothyroidism', u'reduced mobility', u'jaundice', u'lack of energy', u'joint pain', u'vertigo', u'headache', u'tinnitus', u'decrease in appetite', u'insomnia', u'multiple sclerosis', u'dry skin', u'blood in urine', u'increased

In [277]:
symptoms_matched = {}
for d in symptoms:
    drugs = findDrugsForDisease(drug_association_graph,d, chem)
    for drug in drugs:
        symptoms_matched[drug] = {}
        symptoms_matched[drug]['chemName'] = drugs[drug]['name']
        symptoms_matched[drug]['disease'] = d

set([]) set([u'capsaicin']) set([])
set([]) set([u'tenoxicam']) set([])
set([]) set([u'naproxen']) set([])
set([]) set([u'camphor']) set([])
set([]) set([u'caffeine']) set([])
set([]) set([u'modafinil']) set([])
set([]) set([u'orlistat']) set([])
set([]) set([u'topiramate']) set([])
set([]) set([u'magaldrate']) set([])
set([]) set([u'omeprazole']) set([])
set([]) set([u'cimetidine']) set([])
set([]) set([u'famotidine']) set([])
set([]) set([u'esomeprazole']) set([])
set([]) set([u'paracetamol', u'caffeine']) set([u'paracetamol'])
set([]) set([u'nortriptyline']) set([])
set([]) set([u'capsaicin']) set([])
set([]) set([u'denosumab']) set([])
set([]) set([u'gabapentin']) set([])
set([]) set([u'celecoxib']) set([])
set([]) set([u'baclofen']) set([])
set([]) set([u'dextropropoxyphene']) set([])
set([]) set([u'buprenorphine']) set([])
set([]) set([u'bromfenac']) set([])
set([]) set([u'oxandrolone']) set([])
set([]) set([u'aspirin']) set([])
set([]) set([u'tolmetin']) set([])
set([]) set([u't

set([]) set([u'adrenaline']) set([])
set([]) set([u'droperidol']) set([])
set([]) set([u'fructose']) set([])
set([]) set([u'lorazepam']) set([])
set([]) set([u'aprepitant']) set([])
set([]) set([u'granisetron']) set([])
set([]) set([u'perphenazine']) set([])
set([]) set([u'nabilone']) set([])
set([u'carbidopa']) set([]) set([])
set([]) set([u'haloperidol']) set([])
set([]) set([u'mirtazapine']) set([])
set([]) set([u'clozapine']) set([])
set([]) set([u'paracetamol']) set([u'paracetamol'])
set([]) set([u'tetracosactide']) set([])
set([]) set([u'aspirin']) set([])
set([u'ciprofloxacin']) set([]) set([])
set([]) set([u'aspirin']) set([])
set([]) set([u'naproxen']) set([])
set([]) set([u'sulfadiazine']) set([])
set([]) set([u'tetracycline']) set([])
set([]) set([u'erythromycin']) set([])
set([]) set([u'ibuprofen']) set([])
set([]) set([u'colchicine']) set([])
set([]) set([u'hydrochlorothiazide']) set([])
set([]) set([u'torasemide']) set([])
set([]) set([u'nifedipine']) set([])
set([]) set(

In [426]:
symptoms_matched

{'0101010B0': {u'chemName': u'Almasilate', u'disease': u'indigestion'},
 '0101010C0': {u'chemName': u'Aluminium Hydroxide',
  u'disease': u'indigestion'},
 '0101010F0': {u'chemName': u'Magnesium Carbonate',
  u'disease': u'indigestion'},
 '0101010I0': {u'chemName': u'Magnesium Oxide', u'disease': u'indigestion'},
 '0101010J0': {u'chemName': u'Magnesium Trisilicate',
  u'disease': u'indigestion'},
 '0101010M0': {u'chemName': u'Magaldrate', u'disease': u'indigestion'},
 '0101010S0': {u'chemName': u'Calcium Carbonate & Simeticone',
  u'disease': u'indigestion'},
 '0101010T0': {u'chemName': u'Magnesium Sulphate',
  u'disease': u'constipation'},
 '0101012B0': {u'chemName': u'Sodium Bicarbonate', u'disease': u'diarrhea'},
 '0102000AB': {u'chemName': u'Hyoscyamine Sulfate',
  u'disease': u'irritable bowel syndrome'},
 '0102000AD': {u'chemName': u'Pinaverium Bromide',
  u'disease': u'irritable bowel syndrome'},
 '0102000AG': {u'chemName': u'Tegaserod Maleate', u'disease': u'constipation'},
 '0

In [278]:
json.dump(symptoms_matched,open('Symptom_Drug_DrugBank.json','wb'))

In [279]:
# def tokenizeChemName(name):
#     tokens = []
#     spaces = name.lower().split(' ')
#     for s in spaces:
#         subtoken = s.split('/')
#         tokens+=subtoken
#     return tokens
# print tokenizeChemName("aLsaSdjaklsdj/askldjals asdad")

# def looslyMatch(list1 , list2):
#     if len(set(list1).intersection(list2)) > 0:
#         return True
#     else:
#         return False

In [280]:
# DiseaseDrugMatchDict = {}
# for index , row in tqdm(chem.iterrows()):
#     try:
#         normalized_components = tokenizeChemName(row[1].strip())
#     except:
#         print "Found an invalid key"
#         continue
#     print normalized_components
#     for k in diseases:
#         drugs = findDrugsForDisease(drug_association_graph,k ,0.0)
#         drugbank_list = []
#         for d in drugs:
#             drugbank_list+=drugs[d]['name'].split(' ')
            
# #         print normalized_components, drugbank_list
        
#         if len(set(drugbank_list).intersection(normalized_components)) > 0:
#             print "Matched with %s"%k
#             DiseaseDrugMatchDict[row[0]] = {}
#             if 'disease' in DiseaseDrugMatchDict[row[0]]:
#                 DiseaseDrugMatchDict[row[0]]['disease'].append(k)
#             else:
#                 DiseaseDrugMatchDict[row[0]]['disease'] = []
#                 DiseaseDrugMatchDict[row[0]]['disease'].append(k)
#             if 'chemName' not in DiseaseDrugMatchDict[row[0]]:
#                 DiseaseDrugMatchDict[row[0]]['chemName'] = row[1].strip()
                
# print DiseaseDrugMatchDict

In [1425]:
import inspect
import re

def extractPostCodesDict(addrDf):
    postcodeDict = {}
    for index,row in addrDf.iterrows():
        try:
            postcodeDict[row[1]] = row[7].strip()
        except:
            print "Found invalid entry"
    return postcodeDict

def checkIndex(index):
    if index%100 == 0:
        return True
    else:
        return False

def getPC(key , postcodeDict):
    codes = []
    for k in key:
        if k in postcodeDict:
            codes.append(postcodeDict[k])
        else:
            codes.append('')
    return pd.Series(codes,index=key.index)

def getPostcode(df,postcodeDict):
    df[10] = ''
    df[10] = df.groupby(2)[2].apply(getPC,postcodeDict)   
    return df

def getDrugFamily(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = dcode
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDisease(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = diseaseMap[dcode]['disease'].replace('\"','').replace('+',' ')
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDrug(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = diseaseMap[dcode]['chemName']
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDrugPotency(key):
    name = list(set(key))
    if len(name) > 1:
        print("found synonyms")
    text= name[-1]
    found = 0.0
    switch1 = text.find('mcg')
    switch2 = text.find('mg')
    switch3 = text.find('ml')
    
    if switch1 > 0 or switch2 > 0 or switch3 > 0:
        weight = re.findall(r'[0-9]*\.?[0-9]+', text)
        if len(weight) > 0:
            found = max([float(k) for k in weight])
            if switch1 > 0:
                found = found/1000.0
    potency = [found]*len(key)
    return pd.Series(potency,index=key.index)

def countSpecificDrugs(Df, drugs,GPs):
    df_slice = Df.groupby(3)[3].apply(getDrug,drugs)
    selected = df_slice[df_slice!='N/A']
    len(selected)
    df_selected =  Df.iloc[selected.index,:]
    df_selected = df_selected[df_selected[2].isin(GPs.keys())]
    return np.sum(df_selected[5])


def countSpecificDrugCosts(Df, drugs,GPs):
    df_slice = Df.groupby(3)[3].apply(getDrug,drugs)
    selected = df_slice[df_slice!='N/A']
    len(selected)
    df_selected =  Df.iloc[selected.index,:]
    df_selected = df_selected[df_selected[2].isin(GPs.keys())]
    return np.sum(df_selected[6])

def countDrugsByCategoryList(pdp,codes):
    total_drugs = 0.0
    drugs = None
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[5])
    return total_drugs

def countPrescriptionsByCategoryList(pdp,codes):
    total_drugs = 0.0
    drugs = None
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=len(group)
    return total_drugs


def countDrugsCostByCategoryList(pdp,codes):
    total_drugs = 0.0
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[7])
    return total_drugs

def countTotalDrugDosage(pdp,codes):
    total_drugs = 0.0
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[19])
    return total_drugs

def normalizePills(keys):
    minima = np.min(keys)
    if minima > 0:
        potency = keys/minima
    else:
        potency = 1.0
    return pd.Series(potency,index=keys.index)
        
    
def normalizePotency(keys):
    minima = np.min(keys)
    if minima > 0:
        potency = keys/minima
    else:
        potency = 1.0
    return pd.Series(potency,index=keys.index)
    
def normalize(dataFrame):
    dataFrame[16] = dataFrame[3].str[:9]
    dataFrame[17] = dataFrame.groupby(16)[8].apply(normalizePills)
    dataFrame[18] = dataFrame.groupby(16)[15].apply(normalizePotency) 
    dataFrame[19] = dataFrame[18]*dataFrame[17]
    
    
def doImportantMappings(Df, diseaseMap , symptomMap):
    #BNF family
    Df[11] = ''  
    Df[11] = Df.groupby(3)[3].apply(getDrugFamily,symptomMap)
    #Disease
    Df[12] = ''  
    Df[12] = Df.groupby(3)[3].apply(getDisease,diseaseMap)
    #Symptom
    Df[13] = ''  
    Df[13] = Df.groupby(3)[3].apply(getDisease,symptomMap)
    #Checm Name
    Df[14] = ''  
    Df[14] = Df.groupby(3)[3].apply(getDrug,symptomMap)
    #Chem potency
    Df[15] = ''  
    Df[15] = Df.groupby(3)[4].apply(getDrugPotency)

    normalize(Df)

    return Df



In [1426]:
len(chem)

3432

In [1427]:
files

[u'../BL_Work/openPrescribe/uncompressed/T201010PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201902PDPI BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201902CHEM SUBS.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201008ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201601CHEM SUBS.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201601PDPI BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201011PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201901ADDR BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201009PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201011ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201010ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201010CHEM SUBS.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201602ADDR BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201603PDPI BNFT.CSV.tar.gz',
 u'../

In [1428]:
pdp = readPrescriptions('201601',files)
len(pdp)

10036407

In [1429]:
# pdp[:100]

In [1430]:
# pdp[:100].groupby(3)[4].apply(getDrugPotency)

In [1431]:
%store -r Open_filtered_GPs

In [1432]:
pdp_filtered = pdp[pdp[2].isin(Open_filtered_GPs.keys())].copy(deep=True)

In [1433]:
pdp = doImportantMappings(pdp_filtered,DiseaseDrugs,symptoms_matched)

found synonyms
found synonyms


In [1434]:
pdp[:2000]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1962,Q44,01C,N81002,0101010G0BBABAB,Maalox_Susp 195mg/220mg/5ml S/F,1.0,3.35,3.11,500.0,201601.0,,N/A,N/A,N/A,N/A,220.000,0101010G0,500.000000,1.000000,500.000000
1963,Q44,01C,N81002,0101010G0BCABAB,Mucogel_Susp 195mg/220mg/5ml S/F,1.0,2.99,2.77,500.0,201601.0,,N/A,N/A,N/A,N/A,220.000,0101010G0,500.000000,1.000000,500.000000
1964,Q44,01C,N81002,0101021B0AAALAL,Sod Algin/Pot Bicarb_Susp S/F,4.0,30.72,28.42,3000.0,201601.0,,N/A,N/A,N/A,N/A,0.000,0101021B0,3000.000000,1.000000,3000.000000
1965,Q44,01C,N81002,0101021B0AAAPAP,Sod Alginate/Pot Bicarb_Tab Chble 500mg,2.0,9.21,8.53,180.0,201601.0,,N/A,N/A,N/A,N/A,500.000,0101021B0,180.000000,1.000000,180.000000
1966,Q44,01C,N81002,0101021B0BEACAH,Gaviscon_Liq Orig Aniseed Relief,2.0,20.10,18.59,1500.0,201601.0,,N/A,N/A,N/A,N/A,0.000,0101021B0,1500.000000,1.000000,1500.000000
1967,Q44,01C,N81002,0101021B0BEADAJ,Gaviscon Infant_Sach 2g (Dual Pack) S/F,5.0,39.51,36.55,135.0,201601.0,,N/A,N/A,N/A,N/A,0.000,0101021B0,135.000000,1.000000,135.000000
1968,Q44,01C,N81002,0101021B0BEAIAL,Gaviscon Advance_Liq (Aniseed) (Reckitt),23.0,119.11,111.10,11200.0,201601.0,,N/A,N/A,N/A,N/A,0.000,0101021B0,11200.000000,1.000000,11200.000000
1969,Q44,01C,N81002,0101021B0BEAQAP,Gaviscon Advance_Tab Chble Mint(Reckitt),1.0,6.14,5.68,120.0,201601.0,,N/A,N/A,N/A,N/A,0.000,0101021B0,120.000000,1.000000,120.000000
1970,Q44,01C,N81002,0101021B0BEARA0,Gaviscon Double Action_Tab Chble,1.0,3.68,3.41,32.0,201601.0,,N/A,N/A,N/A,N/A,0.000,0101021B0,32.000000,1.000000,32.000000
1971,Q44,01C,N81002,0101021B0BEASA0,Gaviscon Double Action_Liq (Peppermint),1.0,6.47,6.09,500.0,201601.0,,N/A,N/A,N/A,N/A,0.000,0101021B0,500.000000,1.000000,500.000000


In [1435]:
# normalize(pdp)
# pdp.head(100)

In [1436]:
test = pdp[pdp[16]=='0104020N0']
test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
37856,Q44,01R,N81001,0104020N0AAABAB,Kaolin & Morph_Mix,1.0,4.50,4.17,600.0,201601.0,,N/A,N/A,N/A,N/A,0.0,0104020N0,13.333333,1.0,13.333333
225627,Q44,12F,N85051,0104020N0AAABAB,Kaolin & Morph_Mix,1.0,1.50,1.40,200.0,201601.0,,N/A,N/A,N/A,N/A,0.0,0104020N0,4.444444,1.0,4.444444
263218,Q45,00C,A83070,0104020N0AAABAB,Kaolin & Morph_Mix,1.0,1.50,1.40,200.0,201601.0,,N/A,N/A,N/A,N/A,0.0,0104020N0,4.444444,1.0,4.444444
263219,Q45,00C,A83070,0104020N0BEAAAE,Collis Browne's_Mix,2.0,4.98,4.62,90.0,201601.0,,N/A,N/A,N/A,N/A,0.0,0104020N0,2.000000,1.0,2.000000
280647,Q45,00D,A83020,0104020N0AAABAB,Kaolin & Morph_Mix,1.0,2.25,2.19,300.0,201601.0,,N/A,N/A,N/A,N/A,0.0,0104020N0,6.666667,1.0,6.666667
466582,Q45,00M,A81065,0104020N0AAABAB,Kaolin & Morph_Mix,2.0,3.00,2.80,400.0,201601.0,,N/A,N/A,N/A,N/A,0.0,0104020N0,8.888889,1.0,8.888889
548586,Q46,00V,P83007,0104020N0AAABAB,Kaolin & Morph_Mix,2.0,3.00,2.80,400.0,201601.0,,N/A,N/A,N/A,N/A,0.0,0104020N0,8.888889,1.0,8.888889
592343,Q46,00W,P84028,0104020N0AAABAB,Kaolin & Morph_Mix,2.0,3.00,2.80,400.0,201601.0,,N/A,N/A,N/A,N/A,0.0,0104020N0,8.888889,1.0,8.888889
607583,Q46,00W,P84611,0104020N0AAABAB,Kaolin & Morph_Mix,1.0,2.25,2.19,300.0,201601.0,,N/A,N/A,N/A,N/A,0.0,0104020N0,6.666667,1.0,6.666667
796422,Q46,01M,P84042,0104020N0AAABAB,Kaolin & Morph_Mix,1.0,4.50,4.17,600.0,201601.0,,N/A,N/A,N/A,N/A,0.0,0104020N0,13.333333,1.0,13.333333


In [1437]:
min(test[8])

45.0

In [1438]:
print countDrugsByCategoryList(pdp ,opiod_BNF_manual['BNF_code'] )/float(56938269.0)

print countDrugsByCategoryList(pdp ,opiod_BNF['BNF_code'] )/float(56938269.0)

print countPrescriptionsByCategoryList(pdp ,opiod_BNF['BNF_code'] )/float(56938269.0)

print countPrescriptionsByCategoryList(pdp ,opiod_BNF_manual['BNF_code'] )/float(56938269.0)

print countTotalDrugDosage(pdp_filtered,opiod_BNF['BNF_code'] )/56938269.0

print countTotalDrugDosage(pdp_filtered,opiod_BNF_manual['BNF_code'] )/56938269.0

print float(countDrugsCostByCategoryList(pdp_filtered ,opiod_BNF['BNF_code'] ))/56938269.0

print float(countDrugsCostByCategoryList(pdp_filtered ,opiod_BNF_manual['BNF_code'] ))/56938269.0

# print countDrugsByCategoryList(pdp,antiBio_BNF['BNF_code'] )

# 3159501.0/56938269.0

# 56262126.0/3514534.0

0.05855446711946933
0.043398843052288784
0.00611885478991
0.00648767527513
4.5998540108549415
6.30182214591873
0.409705057771
0.499382585375


In [1439]:
all_diabetes_drugs/3514534.0 * 12

10.502512139589488

In [1393]:
float(metforminDrugs)/float(all_diabetes_drugs)

0.49933792876549155

In [1394]:
addr = readAddress('201601',files)
len(addr)

9905

In [1395]:
addr.head()

,0,1,2,3,4,5,6,7
0,201601.0,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
1,201601.0,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
2,201601.0,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,,MIDDLESBROUGH,CLEVELAND,TS1 3BE
3,201601.0,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,,TS14 7DJ
4,201601.0,A81006,TENNANT STREET MEDICAL PRACTICE,TENNANT ST MED PRACT,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AT


In [1396]:
postcodes = extractPostCodesDict(addr)

Found invalid entry


In [1397]:
pdp = getPostcode(pdp , postcodes)

In [1398]:
pdp = pdp.dropna()

In [1399]:
pdp.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
10036059,Q70,99M,J82630,23802098001,H & R_Proshield Plus Skin Prote 115g,1.0,9.84,9.10,1.0,201601.0,GU14 6DH,N/A,N/A,N/A,N/A,0.0,238020980,1.0,1.0,1.0
10036060,Q70,99M,J82630,23803108005,3m Health Care_Cavilon No Sting Barrier,1.0,23.50,21.70,30.0,201601.0,GU14 6DH,N/A,N/A,N/A,N/A,0.0,238031080,30.0,1.0,30.0
10036061,Q70,99M,J82630,23803108006,3m Health Care_Cavilon Durable Barrier C,1.0,8.12,7.51,1.0,201601.0,GU14 6DH,N/A,N/A,N/A,N/A,0.0,238031080,1.0,1.0,1.0
10036062,Q70,99M,J82630,23941009440,Coloplast_Assura B/Plt + 10mm S/H 50mm (,1.0,135.12,124.80,40.0,201601.0,GU14 6DH,N/A,N/A,N/A,N/A,0.0,239410094,8.0,1.0,8.0
10036063,Q70,99M,J82630,23941009666,Coloplast_Assura Insp + H/Away Outl Maxi,1.0,46.89,43.31,30.0,201601.0,GU14 6DH,N/A,N/A,N/A,N/A,0.0,239410096,15.0,1.0,15.0


In [1400]:
%store -r taxonomyDict

In [1401]:
taxonomyDict

{'0702020Z0': {'BNF_Classification': '7.2.2',
  'ChemName': 'Dequalinium Chloride',
  'category': 'Vaginal and Vulval Infections',
  'disease': ''},
 '0202080B0': {'BNF_Classification': '2.2.8',
  'ChemName': 'Bendroflumethiazide/Potassium',
  'category': 'Diuretics With Potassium',
  'disease': ''},
 '0102000L0': {'BNF_Classification': '1.2',
  'ChemName': 'Glycopyrronium Bromide',
  'category': 'Antispasmod.&Other Drgs Alt.Gut Motility',
  'disease': ''},
 '0205040S0': {'BNF_Classification': '2.5.4',
  'ChemName': 'Prazosin Hydrochloride',
  'category': 'Alpha-Adrenoceptor Blocking Drugs',
  'disease': ''},
 '1002010Q0': {'BNF_Classification': '10.2.1',
  'ChemName': 'Pyridostigmine Bromide',
  'category': "Dgs Which Enhance Neuromus'ar Transmi'on",
  'disease': ''},
 '0411000E0': {'BNF_Classification': '4.11',
  'ChemName': 'Rivastigmine',
  'category': 'Dementia',
  'disease': ''},
 '0205051E0': {'BNF_Classification': '2.5.5',
  'ChemName': 'Cilazapril',
  'category': 'Renin-Angiot

In [1402]:
pdp.to_csv('../BL_Work/openPrescribe/serialized/' + '201601' + '.gz',compression='gzip')

In [1403]:
files

[u'../BL_Work/openPrescribe/uncompressed/T201010PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201902PDPI BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201902CHEM SUBS.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201008ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201601CHEM SUBS.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201601PDPI BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201011PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201901ADDR BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201009PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201011ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201010ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201010CHEM SUBS.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201602ADDR BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201603PDPI BNFT.CSV.tar.gz',
 u'../

In [1440]:
DiseaseDrugs

{'0102000AB': {u'chemName': u'Hyoscyamine Sulfate',
  u'disease': u'irritable bowel syndrome'},
 '0102000AD': {u'chemName': u'Pinaverium Bromide',
  u'disease': u'irritable bowel syndrome'},
 '0102000AG': {u'chemName': u'Tegaserod Maleate',
  u'disease': u'irritable bowel syndrome'},
 '0102000AK': {u'chemName': u'Eluxadoline',
  u'disease': u'irritable bowel syndrome'},
 '0103010S0': {u'chemName': u'Ranitidine Bismuth Citrate',
  u'disease': u'rheumatoid'},
 '0103010T0': {u'chemName': u'Ranitidine Hydrochloride',
  u'disease': u'rheumatoid'},
 '0105010E0': {u'chemName': 'Sulfasalazine', u'disease': u'rheumatoid'},
 '0105030A0': {u'chemName': 'Vedolizumab', u'disease': u"crohn's disease"},
 '0106070B0': {u'chemName': 'Linaclotide',
  u'disease': u'irritable bowel syndrome'},
 '0202010L0': {u'chemName': 'Hydrochlorothiazide', u'disease': u'diabetes'},
 '0202080M0': {u'chemName': u'Hydrochlorothiazide/Potassium',
  u'disease': u'diabetes'},
 '020400010': {u'chemName': 'Pindolol With Diure

In [1441]:
monthList = ['201601','201602','201603']
for month in tqdm(monthList):
    pdp = readPrescriptions(month,files)
    pdp = pdp.dropna()
    pdp = getPostcode(pdp , postcodes)
    pdp_filtered = pdp[pdp[2].isin(Open_filtered_GPs.keys())].copy(deep=True)
    pdp_filtered = doImportantMappings(pdp_filtered,DiseaseDrugs,symptoms_matched)
    pdp_filtered.to_csv('../BL_Work/openPrescribe/serialized/' + month + '.gz',compression='gzip')

  0%|          | 0/3 [00:00<?, ?it/s]

found synonyms
found synonyms


100%|██████████| 3/3 [17:51<00:00, 357.01s/it]
